In [2]:
import numpy as np
import os
import cv2
import keras
import math
import tensorflow as tf
import datetime

In [3]:
tf.__version__

'2.18.0'

In [4]:
keras.__version__

'3.6.0'

In [29]:
import cv2
import numpy as np
import math
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout,Flatten

class ShopliftingPrediction:
    def __init__(self, model_path, frame_width, frame_height, sequence_length):
        self.frame_width = frame_width
        self.frame_height = frame_height
        self.sequence_length = sequence_length
        self.model_path = model_path
        self.message = "I will start analyzing the video now."
        self.model = None  # Initialize the model attribute to None

        self.load_model()

    def load_model(self):
        self.model = tf.keras.models.load_model(self.model_path,custom_objects={
            'Conv2D': Conv2D,
            'MaxPooling2D': MaxPooling2D,'Dropout': Dropout,'Flatten':Flatten
        })
    def generate_message_content(self, probability, label):
        if label == 0:
            if probability <= 75:
                self.message = "There is little chance of theft"
            elif probability <= 85:
                self.message = "High probability of theft"
            else:
                self.message = "Very high probability of theft"
        elif label == 1:
            if probability <= 75:
                self.message = "The movement is confusing, watch"
            elif probability <= 85:
                self.message = "I think it's normal, but it's better to watch"
            else:
                self.message = "Movement is normal"

    def Pre_Process_Video(self, current_frame, previous_frame):
        diff = cv2.absdiff(current_frame, previous_frame)
        diff = cv2.GaussianBlur(diff, (3, 3), 0)
        resized_frame = cv2.resize(diff, (self.frame_height, self.frame_width))
        gray_frame = cv2.cvtColor(resized_frame, cv2.COLOR_BGR2GRAY)
        normalized_frame = gray_frame / 255
        return normalized_frame

    def Read_Video(self, filePath):
        self.video_reader = cv2.VideoCapture(filePath)
        if not self.video_reader.isOpened():
            raise ValueError(f"Unable to open video file: {filePath}")
        self.original_video_width = int(self.video_reader.get(cv2.CAP_PROP_FRAME_WIDTH))
        self.original_video_height = int(self.video_reader.get(cv2.CAP_PROP_FRAME_HEIGHT))
        self.fps = self.video_reader.get(cv2.CAP_PROP_FPS)

    def Single_Frame_Predict(self, frames_queue):
        probabilities = self.model.predict(np.expand_dims(frames_queue, axis=0))[0]
        predicted_label = np.argmax(probabilities)
        probability = math.floor(max(probabilities[0], probabilities[1]) * 100)
        return [probability, predicted_label]

    def Predict_Video(self, video_file_path, output_file_path):
        self.Read_Video(video_file_path)
        video_writer = cv2.VideoWriter(output_file_path, cv2.VideoWriter_fourcc(*'mp4v'),
                                       self.fps, (self.original_video_width, self.original_video_height))

        success, frame = self.video_reader.read()
        previous = frame.copy()
        frames_queue = []

        while self.video_reader.isOpened():
            ok, frame = self.video_reader.read()
            if not ok:
                break

            normalized_frame = self.Pre_Process_Video(frame, previous)
            previous = frame.copy()
            frames_queue.append(normalized_frame)

            if len(frames_queue) == self.sequence_length:
                [probability, predicted_label] = self.Single_Frame_Predict(frames_queue)
                self.generate_message_content(probability, predicted_label)
                frames_queue = []

            # Display the message with white border
            cv2.rectangle(frame, (0, 0), (640, 40), (255, 255, 255), -1)
            cv2.putText(frame, self.message, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2, cv2.LINE_AA)
            cv2.putText(frame, self.message, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 1, cv2.LINE_AA)

            video_writer.write(frame)

        self.video_reader.release()
        video_writer.release()


In [17]:
@keras.saving.register_keras_serializable()
class CustomConv2D(Conv2D):
    pass

@keras.saving.register_keras_serializable()
class CustomConv2D(Conv2D):
    pass

In [30]:
m = ShopliftingPrediction(model_path = "lrcn_160S_90_90Q.h5",
                          frame_width = 90, frame_height = 90, sequence_length = 160)
m.load_model()

/home/aarya/stuff/projects/hackathon/mumbai_hack/.venv/lib/python3.12/site-packages/keras/src/layers/core/wrapper.py:27: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


ValueError: Unrecognized keyword arguments passed to LSTM: {'time_major': False}

In [7]:
input_video_file_path = "test5.mp4"
output_video_file_path = 'output.mp4'
m.Predict_Video(input_video_file_path, output_video_file_path)

AttributeError: 'NoneType' object has no attribute 'copy'